In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt2large")
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt2large")
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cuda"
model.to(device)



/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)

In [ ]:
description = "Multilingual generation with mGPT"
def generate(prompt: str, question_num: int):
    # Encode the prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    # Generate text
    generated_ids = model.generate(
        input_ids=input_ids,
        min_length=50,
        max_length=512,
        top_p=0.8,
        top_k=1,
        no_repeat_ngram_size=20,
        pad_token_id=tokenizer.eos_token_id,  # Set pad token to end-of-sequence token
        do_sample=True,  # Enable sampling
        num_return_sequences=1,  # Generate only one sequence
    )

    # Decode generated text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    #print(len(generated_text)-len(prompt), generated_text)
    # Search for the answer to the requested question
    questions = generated_text.split("\n\n  Q: ")
    for i , q in enumerate(questions):
        if i == question_num:
            return q.split("\n  A: ")[1]

    return "Answer not found"
       



In [ ]:
def generate(prompt: str):
        input_ids = tokenizer.encode(prompt, return_tensors="pt").cuda(device)
        out = model.generate(input_ids, 
                min_length=10, 
                max_length=1024, 
                top_p=0.8, 
                top_k=1,
                do_sample=True,  # Enable sampling
                no_repeat_ngram_size=20
        )
        end = "<|endoftext|>"
        generated_text = list(map(tokenizer.decode, out))[0]
        endoftext_idx = generated_text.find(end)
        if endoftext_idx >= 0:
                generated_text = generated_text[:endoftext_idx]
        return generated_text


prompt_hu = """Te egy tapasztalt bot vagy aki kérdésekre válaszol.

Kérdés: Hol van Budapest?
Válasz:

"""
print(generate(prompt_hu))

In [ ]:
import re
from typing import List
from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/mGPT")
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/mGPT")
device = "cuda"
model.to(device)

In [ ]:
def generate(context: str, questions: List[str]) -> str:
    # Concatenate the context and questions
    prompt = context + '\n\n' + '\n\n'.join([f'{i+1}. {q}' for i, q in enumerate(questions)])
    # Encode the prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    # Generate text
    generated_ids = model.generate(
        input_ids=input_ids,
        min_length=10,
        max_length=1024,
        top_p=0.8,
        top_k=1,
        no_repeat_ngram_size=20,
        pad_token_id=tokenizer.eos_token_id,  # Set pad token to end-of-sequence token
        do_sample=True,  # Enable sampling
        num_return_sequences=1,  # Generate only one sequence
    )

    # Decode generated text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print(generated_text)
    # Split generated text by the question numbers and return a dictionary
    answers = {}
    for i, q in enumerate(questions):
        # Find the answer text between the question number and the next question number
        start = f'{i+1}. {q}\n\n'
        end = f'{i+2}. ' if i < len(questions) - 1 else ''
        answer_text = generated_text.split(start)[-1].split(end)[0]
        answers[q] = answer_text.strip()
    return answers

In [ ]:
context = "A termék ára 1024 Ft, színe piros. Jövő héten lesz .Az alábbiakban válaszolunk a felmerült kérdésekre:\n1. Mikor lesz elérhető az új termék?\n2. Milyen színekben lesz elérhető?\n3. Mennyibe fog kerülni az új termék?"
question_1 = "Mikor lesz elérhető az új termék?"
question_2 = "Milyen színekben lesz elérhető?"
question_3 = "Mennyibe fog kerülni az új termék?"

# GPT-2 model válaszgenerálás
answers = generate(context, [question_1, question_2, question_3])
print(answers)
# A válaszok és a kérdések összerendelése az azonosítók alapján
answer_dict = {}
for answer in answers:
    match = re.match(r"(\d+)\. (.+)", answer)
    if match:
        question_num = int(match.group(1))
        answer_text = match.group(2)
        answer_dict[question_num] = answer_text

# Az összerendelt válaszok kiírása a kérdésekhez
print("Válaszok:")
print(f"1. {answer_dict[1]}")
print(f"2. {answer_dict[2]}")
print(f"3. {answer_dict[3]}")


In [ ]:
# https://huggingface.co/models?pipeline_tag=question-answering&language=hu&sort=downloads

In [ ]:
context="A szabadság jó"
question=  "Milyen a szabadság?" 

In [ ]:
from transformers import pipeline
qa_pipeline = pipeline(
  "question-answering",
  model = "ZTamas/hubert-qa-milqa-impossible-long-answer",
  tokenizer = "ZTamas/hubert-qa-milqa-impossible-long-answer",
  device = 0,                      #GPU selection, -1 on CPU
  handle_impossible_answer = True,
  max_answer_len = 1000            #This can be modified, but to let the model's
                                   #answer be as long as it wants so I
                                   #decided to add a big number
  )

predictions = qa_pipeline({
    'context': context,            
    'question': question           
})

print(predictions)
